# Classification de texte avec TF-IDF et RandomForest

Ce notebook détaille un pipeline complet de classification de texte utilisant le prétraitement linguistique avec SpaCy, la vectorisation TF-IDF et un modèle RandomForest. Chaque étape est expliquée, du nettoyage à la génération du fichier de soumission.

---

## Concepts clés

- **TF-IDF (Term Frequency-Inverse Document Frequency)** : transforme chaque texte en un vecteur de nombres, reflétant l'importance des mots dans le corpus.
- **RandomForestClassifier** : algorithme d'ensemble basé sur la combinaison de plusieurs arbres de décision, robuste et efficace pour la classification.
- **Prétraitement linguistique** : nettoyage, lemmatisation, suppression des stopwords pour améliorer la qualité des features.

---

In [ ]:
# Importation des bibliothèques nécessaires
import json
import pandas as pd
import numpy as np
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## 1. Chargement et préparation des données d'entraînement

On charge les données d'entraînement depuis un fichier JSON. On remplace les valeurs manquantes dans la colonne 'description' par une chaîne vide pour éviter les erreurs lors du traitement.

In [ ]:
with open('train.json', 'r', encoding='utf-8') as f:
    train_data = json.load(f)
df_train = pd.DataFrame(train_data)
df_train['description'] = df_train['description'].fillna("")

## 2. Chargement des labels

On associe à chaque texte sa catégorie cible pour l'apprentissage supervisé, en fusionnant avec le fichier de labels.

In [ ]:
labels_df = pd.read_csv('train_label.csv')  # colonnes : Id, Category
df_train = df_train.merge(labels_df, on='Id')

## 3. Initialisation de SpaCy et fonction de nettoyage + lemmatisation

On utilise SpaCy pour nettoyer et lemmatiser les textes :
- Suppression du HTML et des URLs
- Mise en minuscules
- Lemmatisation (réduction à la racine)
- Suppression des stopwords et tokens non alphabétiques

Ce prétraitement réduit la dimensionnalité et améliore la qualité des features pour le modèle.

In [ ]:
nlp = spacy.load('en_core_web_lg')

def clean_and_tokenize(text):
    if not isinstance(text, str) or not text.strip():
        return ""
    text = re.sub(r'<[^>]+>', ' ', text)
    text = re.sub(r'http\S+', ' ', text)
    text = text.lower().strip()
    doc = nlp(text)
    tokens = [tok.lemma_ for tok in doc if tok.is_alpha and not tok.is_stop]
    return " ".join(tokens)

df_train['Clean'] = df_train['description'].apply(clean_and_tokenize)

## 4. Vectorisation TF-IDF sur l'entraînement

On transforme les textes en vecteurs de nombres avec `TfidfVectorizer` :
- Chaque texte devient un vecteur de taille `max_features` (ici 20 000)
- Chaque dimension correspond à un mot ou une paire de mots (n-grammes)
- La valeur est le score TF-IDF du mot dans le texte

Mathématiquement, le TF-IDF d'un mot $w$ dans un document $d$ est :
$$ \text{tfidf}(w, d) = tf(w, d) \times \log\left(\frac{N}{df(w)}\right) $$
où $tf(w, d)$ est la fréquence du mot dans le document, $N$ le nombre total de documents, $df(w)$ le nombre de documents contenant $w$.

In [ ]:
tfidf = TfidfVectorizer(
    max_features=20000,
    ngram_range=(1,2),
    min_df=5,
    norm='l2'
)
X_tfidf = tfidf.fit_transform(df_train['Clean'])
y = df_train['Category']

## 5. Split train / validation

On sépare les données en un ensemble d'entraînement (80%) et de validation (20%) pour évaluer la capacité de généralisation du modèle. On stratifie pour conserver la proportion des classes.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X_tfidf, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

## 6. Entraînement du modèle RandomForest

Le RandomForest est un ensemble d'arbres de décision. Chaque arbre vote pour une classe, et la classe majoritaire est choisie. Cela permet de réduire le surapprentissage et d'améliorer la robustesse.

- `n_estimators=200` : nombre d'arbres
- `max_depth=30` : profondeur maximale des arbres
- `n_jobs=-1` : utilise tous les cœurs du processeur pour accélérer l'entraînement

Mathématiquement, chaque arbre partitionne l'espace des features, et la forêt combine les décisions pour une meilleure généralisation.

In [ ]:
rf_clf = RandomForestClassifier(
    n_estimators=200,
    max_depth=30,
    random_state=42,
    n_jobs=-1
)
rf_clf.fit(X_train, y_train)

## 7. Évaluation sur la validation

On prédit les classes sur l'ensemble de validation et on affiche un rapport de classification (précision, rappel, F1-score).

In [ ]:
y_pred = rf_clf.predict(X_val)
print("=== Rapport de classification sur la validation ===")
print(classification_report(y_val, y_pred))

## 8. Chargement et préparation des données de test

On applique le même nettoyage et la même vectorisation aux textes de test.

In [ ]:
with open('test.json', 'r', encoding='utf-8') as f:
    test_data = json.load(f)
df_test = pd.DataFrame(test_data)
df_test['description'] = df_test['description'].fillna("")
df_test['Clean'] = df_test['description'].apply(clean_and_tokenize)

## 9. Vectorisation TF-IDF sur le test

On transforme les textes de test en vecteurs TF-IDF avec le même vocabulaire que l'entraînement.

In [ ]:
X_test = tfidf.transform(df_test['Clean'])

## 10. Prédiction

On prédit la catégorie pour chaque texte de test.

In [ ]:
preds = rf_clf.predict(X_test)

## 11. Préparation du fichier de soumission

On prépare le fichier de soumission au format attendu, associant chaque Id de test à la catégorie prédite. On s'assure de l'alignement sur l'Id.

In [ ]:
template = pd.read_csv('template_submissions.csv')  # colonnes : Id, Category
mapping = dict(zip(df_test['Id'], preds))
template['Category'] = template['Id'].map(mapping)

## 12. Sauvegarde

On sauvegarde le fichier de soumission au format CSV.

In [ ]:
output_path = 'submission_rf.csv'
template.to_csv(output_path, index=False)
print(f"Fichier de soumission enregistré : '{output_path}'")

---

## Concepts mathématiques et conclusion

- **TF-IDF** : pondère l'importance des mots selon leur fréquence dans le document et leur rareté dans le corpus.
- **RandomForest** : combine plusieurs arbres de décision pour améliorer la robustesse et la généralisation.
- **Classification** : la sortie du modèle est la classe avec la majorité des votes des arbres.

Ce pipeline montre comment passer de textes bruts à des prédictions de classes avec un modèle d'ensemble, en utilisant des représentations vectorielles robustes.